Imports

In [1]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

Dataset preparation

DATASET CLASS

In [2]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None, mask_transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms
        self.mask_transforms = mask_transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                mask_filename = f"{image_filename}_SegmentationClass.png"
                mask_path = os.path.join(root_msk, mask_filename)
                if os.path.exists(mask_path):
                    self.pairs.append((image_path, mask_path))

        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)
        self.input_channels = image_array.shape[0]

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
        
        #mask = Image.open(mask_path)
        #mask_array = np.array(mask)
        mask_array = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        self.input_channels = mask_array.shape[0]

        # Apply transformations
        if self.mask_transforms:
            mask_array = self.mask_transforms(mask_array)


        return image_array, mask_array
        


DATASET SLPIT

In [3]:
import random
from torch.utils.data import random_split
from torchvision.transforms import functional as F

# magari rifare il dataloader con due cartelle
image_path = '/kaggle/input/rgbdataset/RGB_dataset'
mask_path = '//kaggle/input/512-images/Resizedmasks_smaller'

#normalize = transforms.Normalize(mean=[0.5], std=[0.5])
#normalize = transforms.Normalize(mean=[35.5, 35.2, 33.4], std=[21.8, 21.6, 20.9])

transform = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize(mean=[35.5, 35.2, 33.4], std=[21.8, 21.6, 20.9]),
    # Add other transforms here as needed
])

mask_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    # Add other mask transformations here
])


# Create the dataset
dataset = ParkingLotDataset(image_path, mask_path, transforms=transform, mask_transforms=mask_transforms )

# Define the proportions for the split
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

total_size = len(dataset)
print(total_size)

# Shuffle the dataset
random.shuffle(dataset.pairs)

# Calculate the sizes of each split
total_size = len(dataset)
print(total_size)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_pairs = dataset.pairs[:train_size]
val_pairs = dataset.pairs[train_size:train_size+val_size]
test_pairs = dataset.pairs[train_size+val_size:]

# Create datasets for each split
train_dataset = ParkingLotDataset(image_path, mask_path, pairs=train_pairs, transforms=transform)
val_dataset = ParkingLotDataset(image_path, mask_path, pairs=val_pairs, transforms=transform)
test_dataset = ParkingLotDataset(image_path, mask_path, pairs=test_pairs, transforms=transform )

# Now you can create data loaders for each split
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False) 


500
500


In [7]:
import matplotlib.pyplot as plt

# Assuming you have a DataLoader object called train_loader
num_samples = 10

# Iterate over batches from the DataLoader
for batch_idx, (images, masks) in enumerate(train_loader):
    # Get the number of samples in the current batch
    batch_size = images.size(0)

    # Iterate over the samples in the batch
    for i in range(batch_size):
        # Convert the image tensor to a NumPy array
        image_array = images[i].permute(1, 2, 0).numpy()

        # Convert the mask tensor to a NumPy array
        mask_array = masks[i].permute(1, 2, 0).numpy()

        # Plot the image and the combined mask
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(image_array, cmap='gray')
        plt.title(f'Image {batch_idx * batch_size + i + 1}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(image_array, cmap='gray')
        plt.imshow(mask_array, cmap='gray', alpha=0.5)
        plt.title(f'Image {batch_idx * batch_size + i + 1} with Masks')
        plt.axis('off')

        plt.show()

        # Break after the first 10 samples
        if batch_idx * batch_size + i == num_samples - 1:
            break

    # Break after processing one batch
    if batch_idx == 0:
        break

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3

Model

In [5]:
import torch
from torchvision import models
from torch import nn
import torch.nn.functional as F

class SmallUNet(nn.Module):
    def __init__(self):
        super(SmallUNet, self).__init__()
        
        # Encoder
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  #scala di grigi
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # Modificato il numero di canali di input a 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.maxpool = nn.MaxPool2d(2)
        
        # Decoder
        self.upconv3 = nn.Conv2d(384, 128, 3, padding=1)
        self.upconv2 = nn.Conv2d(192, 64, 3, padding=1)
        self.upconv1 = nn.Conv2d(96, 32, 3, padding=1)
        self.final_conv = nn.Conv2d(32, 1, 1)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        # Encoder
        conv1 = F.relu(self.conv1(x))
        x = self.maxpool(conv1)
        
        conv2 = F.relu(self.conv2(x))
        x = self.maxpool(conv2)
        
        conv3 = F.relu(self.conv3(x))
        x = self.maxpool(conv3)
        
        x = F.relu(self.conv4(x))
        
        # Decoder
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = F.relu(self.upconv3(x))
        
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = F.relu(self.upconv2(x))
        
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = F.relu(self.upconv1(x))
        
        out = torch.sigmoid(self.final_conv(x))
        
        return out
    

Train

In [6]:
from torch.utils.data import DataLoader
from torch import optim
import torch.nn.functional as F
import wandb

# Start a new run
#run = wandb.init(project='Parking_lot_zones', entity='Fede_occe')




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SmallUNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
#wandb.watch(model)
epochs = 50

for epoch in range(epochs):
    model.train()
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        masks = masks.unsqueeze(1).type_as(outputs)  # Add a channel dimension to the masks
        loss = F.binary_cross_entropy_with_logits(outputs, masks)
        loss.backward()
        optimizer.step()

        #wandb.log({"Train Loss": loss.item()})

    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            masks = masks.unsqueeze(1).type_as(outputs)
            val_loss = F.binary_cross_entropy_with_logits(outputs, masks)

            #wandb.log({"Validation Loss": val_loss.item()})

    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

Epoch 1, Training Loss: -27.742427825927734, Validation Loss: -29.41171646118164
Epoch 2, Training Loss: -27.746206283569336, Validation Loss: -29.41555404663086
Epoch 3, Training Loss: -27.747289657592773, Validation Loss: -29.41667938232422
Epoch 4, Training Loss: -27.747791290283203, Validation Loss: -29.417198181152344
Epoch 5, Training Loss: -27.74806785583496, Validation Loss: -29.417495727539062


KeyboardInterrupt: 